# ChainerのExample [MNIST]
* chainer公式チュートリアルのexampleを書く
* http://docs.chainer.org/en/stable/tutorial/basic.html

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, Variable, optimizers, serializers, utils
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L

In [3]:
import data

## MNISTデータのダウンロード
* 28×28ピクセル、[0,1]の値に変換

In [4]:
mnist = data.load_mnist_data()

Done
Done
Done
Done
Converting training data...
Done
Converting test data...
Done
Save output...
Done
Convert completed


In [5]:
x_all = mnist['data'].astype(np.float32) / 255
y_all = mnist['target'].astype(np.int32)
x_train, x_test = np.split(x_all, [60000])
y_train, y_test = np.split(y_all, [60000])


In [7]:
print x_all.shape
print x_train.shape
print x_test.shape

(70000, 784)
(60000, 784)
(10000, 784)


## モデルの定義
* 4層の線形NNモデルを作る

In [8]:
class MLP(Chain):
    def __init__(self):
        # 各層でのlink(ネットワークのパラメータ)の定義
        super(MLP, self).__init__(
            l1=L.Linear(784, 100),
            l2=L.Linear(100, 100),
            l3=L.Linear(100, 10),
        )
        
    def __call__(self, x):
        # chainの定義
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        y = self.l3(h2)
        return y

## ClassifierChainの定義
* 損失関数の計算、予測精度の計算をするためのクラス
* Classifierもchainとして書く
* chainer.links.Classifierとして実装されているので、以下で書いたのはその下では使わない

In [17]:
class Classifier(Chain):
    def __init__(self, predictor):
        super(Classifier, self).__init__(predictor=predictor)
    
    def __call__(self, x, t):
        y = self.predictor(x)
        self.loss = F.softmax_cross_entropy(y, t)
        self.accuracy = F.accuracy(y, t)
        return self.loss

## Optimizerの定義
* 上で書いたClassifierクラスがlinksモジュール下に実装されているので、それを使う
* OptimizerにはSGDを使う

In [9]:
model = L.Classifier(MLP())
optimizer = optimizers.SGD()
optimizer.setup(model)

## 学習ループ

In [32]:
batchsize = 100
datasize = 60000
for epoch in range(20):
    sum_accuracy = 0
    sum_loss = 0
    print('epoch %d' % (epoch))
    indexes = np.random.permutation(datasize)
    for i in range(0, datasize, batchsize):
        x = Variable(x_train[indexes[i : i + batchsize]])
        t = Variable(y_train[indexes[i : i + batchsize]])
        
        ## 
        #optimizer.update(model, x, t)
        
        ## 勾配を明示的に計算する場合
        model.zerograds()
        loss = model(x, t)
        loss.backward()
        optimizer.update()
        
        sum_loss += loss.data * len(t.data)
        sum_accuracy += model.accuracy.data * len(t.data)
    print('loss:%s,  acc:%s'%(sum_loss/datasize, sum_accuracy/datasize))


epoch 0
loss:0.0120115954902,  acc:0.998616667986
epoch 1
loss:0.0119287882479,  acc:0.998533334732
epoch 2
loss:0.0116715136427,  acc:0.998633334637
epoch 3
loss:0.0115923802013,  acc:0.998683334589
epoch 4
loss:0.0114978061043,  acc:0.998766667843
epoch 5
loss:0.0113368175113,  acc:0.998616667986
epoch 6
loss:0.0112133060052,  acc:0.998666667938
epoch 7
loss:0.0110510870995,  acc:0.998750001192
epoch 8
loss:0.0109385950762,  acc:0.998766667843
epoch 9
loss:0.0108498769942,  acc:0.998816667795
epoch 10
loss:0.0107161713192,  acc:0.998866667747
epoch 11
loss:0.010520609152,  acc:0.998816667795
epoch 12
loss:0.0104453810793,  acc:0.998933334351
epoch 13
loss:0.0103325763275,  acc:0.999066667557
epoch 14
loss:0.010146433936,  acc:0.998966667652
epoch 15
loss:0.0100609428678,  acc:0.998966667652
epoch 16
loss:0.00997522459327,  acc:0.998983334303
epoch 17
loss:0.00984644341321,  acc:0.999016667604
epoch 18
loss:0.0097248304774,  acc:0.999033334255
epoch 19
loss:0.0095872351209,  acc:0.999

In [28]:
loss.data

array(0.03637012466788292, dtype=float32)

In [16]:
model.loss.data

array(0.11869700253009796, dtype=float32)

In [19]:
len(t.data)

100

## 学習状況の評価

In [25]:
sum_loss, sum_accuracy = 0, 0
for i in range(0, 10000, batchsize):
    x = Variable(x_test[i : i + batchsize])
    t = Variable(y_test[i : i + batchsize])
    loss = model(x, t)
    sum_loss += loss.data * batchsize
    sum_accuracy += model.accuracy.data * batchsize

mean_loss = sum_loss / 10000
mean_accuracy = sum_accuracy / 10000
print('loss : %s,  acc : %s'%(mean_loss, mean_accuracy))

loss : 0.0791966257518,  acc : 0.976600005627
